# コンテンツ
1. ぶっとばしディープラーニング(DL)
2. ぶっとばし強化学習(RL)
3. DL×RLのフレームワークで自然言語処理(テキスト生成)を扱う

# １ ぶっとばしディープラーニング
## 重要概念
- ネットワーク構造の具体例
- 評価関数
- パラメータ最適化

## ネットワーク構造の具体例
![](../images/dn.png)

### リニア層
- 別名：dense層、アフィン層、FC(fully connected)層
- 実体はアフィン変換(線形変換＋平行移動)
    - 入力次元を拡張するとアフィン変換は線形変換とみなすことができる
    - $\begin{pmatrix} h1_0 \\ h1_1 \\ h1_2 \end{pmatrix} = 
    \begin{pmatrix} \theta_{0,0} & \theta_{0,1} & \theta_{0,2} & \theta_{0,3} \\ \theta_{1,0} & \theta_{1,1} & \theta_{1,2} & \theta_{1,3}
    \\ \theta_{2,0} & \theta_{2,1} & \theta_{2,2} & \theta_{2,3} \end{pmatrix}\begin{pmatrix} x_0 \\ x_1 \\ x_2 \\ 1 \end{pmatrix}$
    
    
- 入力層は3つのユニット。($x_0$, $x_1$,$x_2$,1)という4つの実数の組
- 出力層は3つのユニット。($h1_0$,$h1_1$, $h1_2$)という3つの実数の組
- パラメータは$\Theta$は12個のパラメータの組。
- リニア層はパラメータ$\Theta$で特徴づけられている
- 学習時はこのパラメータの値が変化していく

### ノンリニア層
- gelu,softmax変換など
- ネットワークに大きな表現力をもたせるために必要
- 可変パラメータを持たない

### 全体
- 層間ごとの変換を合成したものが全体の変換になる
    - $\boldsymbol y = f(\boldsymbol x) = softmax(f_2(f_1(\boldsymbol x)))$
    - たくさんの層をつくることで、同じ表現力をもつ関数をより少ないパラメータ数でつくることができる→ディープラーニングすごいの由来

### GPT系のネットワーク
- GPTの一番最初の層はembedding層と言われている。これは総トークン数をにいれられて1600次元のベクトルに圧縮される。つまり$\Theta_{emb}$は(1600, 50257)の行列であり、その実体は1億個弱のパラメータ。１つのトークンごとにこの変換がおこなわれる。
- 変換は複数のベクトルをまとめて処理する。たとえばアテンション層の入力は(n_batch, n_token, n_dim)の形をもつランク3のテンソル
- n_dim=1600が上記の入力層のユニット数に相当する


## 評価関数
- そのネットワークがどれだけいけてるかを評価するための関数。
- 教師あり学習においてはネットワークの出力$\boldsymbol y$(予測値)と教師データ$\boldsymbol t$(実測値)と照らし合わせてネットワークの性能を評価する
- たとえば予測タスクにおいては、評価関数として以下のような損失関数を用いる
$Ｌ_{\Theta}(\boldsymbol y, \boldsymbol t)$
- Ｌの値が0に近いときはネットワークの出力$y$はいい感じ、大きいときはダメとみなせる
- $\boldsymbol y$はパラメータ$\Theta$で特徴づけられたネットワークの出力である($\boldsymbol y=f_{\Theta}(\boldsymbol x)$)ため損失関数の値自体も$\Theta$で特徴づけられる
- 与えられたどんなデータに対してもＬの値が小さくなるようにネットワークパラメータ$\Theta$を決定することが望ましい

- あるいはもっとシンプルに出力$y$のイケてる度合いをストレートに表現する評価関数としては報酬関数$R$がある
$R_{\Theta}(\boldsymbol y)$
- Rの値が大きければ大きいほどネットワークの出力$y$はいい感じとみなせる
- 与えられたどんなデータに対しても$R$の値が大きくなるようにネットワークパラメータ$\Theta$を決定することが望ましい


## パラメータ最適化
- 評価関数において、最高の$\Theta$をみつけることをパラメータ最適化という
- データを変数とみなす見方からパラメータを変数とみなす見方への変換が大事
    - $L_{\Theta}(\boldsymbol y, \boldsymbol t)  \rightarrow  L(\Theta)$

最適化は２つのフェイズにわけることができる

### フェイズ1 すべてのパラメータの現在の勾配を導出して修正の方向性を決定する
- 逆伝播(バックプロップ)。詳細略。
- 大事なこと：損失関数$L$がパラメータ$\Theta$の全ての要素で微分可能でないと学習できない。
- 変換$f_1$や$f_2$は線形変換なので微分可能
- 非線形変換は油断したら微分可能ではなくなることもある
- 出力にランダムサンプリングがおこなわれると微分可能でなくなる

### フェイズ2 実際にパラメータをその方向へ修正する
- 方向はわかったけど大きさはどれくらい修正すればいいの問題
- 学習率$\eta$の導入
    - $\Theta \leftarrow \Theta - \eta \nabla_{\Theta} L(\Theta)$
    - 学習初期は$\eta$は大きめで大胆に修正していく。後半は小さくして微調整する感じ。


# ぶっとばし強化学習
![世界観](../images/rl_env.png)
- 強化学習の目的は<エージェント>が<環境>からいい感じの<報酬>を得られるような<行動>をとることができるような<ポリシー>を見つけること
- 機械学習の文脈で語られがちだが、本来は関係のない文脈で発展してきた。

## 重要概念
- エージェント、環境、状態$s$、行動$a$、報酬$r$、ポリシー$\pi$


### エージェントと環境
- エージェントは主役。
- 環境はエージェントとコミュニケーションをとる対象。世界のインターフェイスみたいなもの。世界についての情報はすべて環境をつうじてやりとりされる。

### 状態と行動
- $s_1,s_2,s_3,s_4,,,s_n$という有限個の状態があり、ある行動をとるとひとつの状態から別の状態へ遷移する
- たとえば将棋では何かの手を指すと盤面が変化する。この場合、状態はあらゆる盤面の集合の要素(スナップショット)であり、行動は具体的な差し手である。この場合、行動も有限個の要素の集合になる。
- 「行動」を定式化すると$s_{t+1} = a(s_t)$となる

| エージェント   | 環境        | 状態              | 行動           |
|---------------|------------|-------------------|----------------|
|将棋AI　　　　  | 盤面      　|譜面               |6五銀           |
|二足歩行ロボット|物理フィールド|障害物の位置ベクトル|ジャンプ         |
|chatBot        |ユーザー     |人間の入力テキスト  |Botの出力テキスト |

### ポリシーと状態遷移
- ポリシーは「こういう状態のときはこういう行動をとる」という決め事。条件付き確率分布であらわす。
- たとえば、「状態が$s_1$のときは1/3の確率で$a_1$、2/3の確率で$a_2$の行動をする」というポリシーの場合は、
    - $\pi(A=a_1|S=s_1) = 1/3$
    - $\pi(A=a_1|S=s_1) = 2/3$

強化学習の目的はいけてるポリシーを見つけること。
- テキスト生成においては入力テキストSを条件づけたときの出力ロジットを決定するネットワークそのものがまさにポリシーである。

![状態の例](../images/rl_state_example.png)
![状態遷移](../images/state_transition.png)

### 報酬
- なにかの行動をとったときにそれがどれだけアツいものだったかを示す実数指標(負の値のときは逆にイケてない行動だったということ)。
- 報酬は行動後に環境から与えられる。具体的には報酬は状態と行動の関数で表現する。
    - $ r = f(s,a)$ 
    - 状態$S_1$のときに行動$a_1$をとると報酬rを得ることができる。

In [ ]:
# 疑似コード
class WorldInterface:
    pass

class Policy:
    def __init__(self, params):
        self.params = params

    def __call__(self, actions):
        # choose one from actions algrithm
        return action
    
    def update(self):
        pass

class Agent:
    def __init__(self, initial_policy: Policy):
        self.policy = initial_policy
        self.actions = range(10)
        self.rewards: list[int]
    
    def get_reward(self, state, next_state, action) -> int:
        # define reward function here
        self.rewards.append(reward)
        return self.rewards[-1]
    
    def get_action(self, state):
        return self.policy()

    def update_policy(self, rewards):
        self.policy.update()

if __name__ == "__main__":
    agent = Agent(Policy())
    env = WorldInterface()
    epoch = 10
    state = env.get_current_state()
    for _ in range(epoch):
        action = agent.get_action(state)
        env.update_state()
        next_state = env.get_current_state(action)
        reward = agent.get_reward(state, next_state, action)
        state = next_state
    
    agent.update_policy()

### 軌跡と方策勾配法
- 初期状態$s_0$からポリシー$\pi$にしたがって、行動をランダムサンプリングしていくとき、シーケンス
$(s_0, a_0, r_0, s_1, a_1, r_1,.......,s_{n-1}, a_{n-1}, r_{n-1}, s_n)$
を軌跡という。
- 方策勾配法：軌跡の報酬合計の期待値を評価関数として設定し、これが最大になるようなポリシーをみつける手法
    - 状態$s_i$や行_動$a_i$が確率変数なのでいろんな軌跡をたどるため、報酬$r_i$も確率変数になる
    - そのために評価関数を決定するためには報酬の期待値をとる必要がある

### 強化学習の目的(再び)
- 報酬をいっぱいもらえるような行動をとることができるようにポリシーを変更していくこと
- このあいまいなニュアンスの日本語を定式化するのにいろいろな手法がある
- いろいろな手法の詳細は略する

# DL×RLのフレームワークで自然言語処理を扱う
## テキスト生成タスクに強化学習の文脈をあてはめると、、
![状態遷移](../images/state_transition_llm.png)
- エージェントはチャットボットであり、目的は報酬をたくさんうけとること
- 状態$S$は相手から生成されるテキスト
- 行動$a$は自分(チャットボット)が生成する文字
- ポリシー$\pi(a|s)$は条件付き確率分布である
- 行動$a$は1トークンのみのため、これを繰り返すことで軌跡を形成する
- ある程度の出力が終わったときに報酬$r$が確定する
- ユーザーからの返信によって行動$a$がよかったのかどうかを判別する
- つまりユーザーのお気に召すような行動をとるように学習するのが強化学習


## テキスト生成タスクにディープラーニングの文脈をあてはめると、、
1. ポリシー$\pi$はニューラルネットワークで表現された関数である
    - これはGPT(LMHead)の出力であるところのロジットにソフトマックスを適用したものに相当する
    - 入力に文字列、出力に次に生成される文字をとる
    - ポリシーのパラメータ$\Theta$を更新していく
2. 報酬関数もニューラルネットワークで表現された関数である
    - 報酬関数のパラメータの学習は教師あり学習である
    - 教師ありデータが俗にいうヒューマンフィードバックである
    - 報酬関数rはユーザーの嗜好を反映する形で定義される


## まとめ
![モデルフロー](../images/model_flow.png)

### 説明
- ポリシー$\pi$は入力にトークン(列)$s$をとり、出力行動$a$に次トークンのロジットを出力する言語モデルである。つまり$p=\pi(a|s)$
- 状態遷移は$s' = p(s,a) = s + x':x~\pi$
- 報酬は1回の行動ごとに発生するわけではなく、ある程度トークンが出力されてはじめて発生する。最大出力トークン数を超えると発生する
- つまり軌跡(trajectory)$(s_0, a_0, r_0, s_1, a_1, r_1,.......,s_{n-1}, a_{n-1}, r_{n-1}, s_n)$ において$r_0=r_1=...=r_{n-1}=0$である。
- 報酬関数$R$は入力にトークン(列)をとり、出力に任意の実数をひとつ出力する言語モデルである。分析ヘッドは1層のリニア層((1, n_embed)の行列)である。

### 次回
- このモデルフローを題材にして、どうやってポリシー$\pi$を学習していくのかという話
- 具体的には評価関数をどうする？
- PPO(proximal policy optimization)をとりあげる予定

# 参考文献
- [ディープラーニングを支える技術〈2〉 ——ニューラルネットワーク最大の謎](https://www.amazon.co.jp/dp/429712811X)
- [ゼロから作るDeep Learning 4](https://www.amazon.co.jp/dp/4873119758)
    - 主に9章の方策勾配法
- [Proximal Policy Optimization Algorithms](https://arxiv.org/pdf/1707.06347.pdf)